# OpenSearch Playground

## Import packages

In [8]:
from opensearchpy import OpenSearch, helpers
from faker import Faker
from faker.providers import internet, date_time

## Create Client

In [6]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin')
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compression = True,
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_show_warn = False
)
health = client.cat.health()
print(health)


1646816660 09:04:20 opensearch-cluster green 2 2 true 10 5 0 0 0 0 - 100.0%



## Define constants

In [ ]:
template_name = 'logs-hss'
template_body = {
    'index_patterns': [
        'logs-hss'
    ],
    'data_stream': {},
    'priority': 100
}

## Create index template and data stream

In [7]:
create_index_template_result = client.indices.put_index_template(template_name, template_body)
get_index_template_result = client.indices.get_index_template(template_name)
print(get_index_template_result)

create_data_stream_result = client.indices.create_data_stream(template_name)
get_data_stream_result = client.indices.get_data_stream(template_name)
print(get_data_stream_result)

{'index_templates': [{'name': 'logs-hss', 'index_template': {'index_patterns': ['logs-hss'], 'priority': 100, 'data_stream': {'timestamp_field': {'name': '@timestamp'}}}}]}
{'data_streams': [{'name': 'logs-hss', 'timestamp_field': {'name': '@timestamp'}, 'indices': [{'index_name': '.ds-logs-hss-000001', 'index_uuid': 'c99Mg1rwTeu-jJmFqH8eRg'}], 'generation': 1, 'status': 'YELLOW', 'template': 'logs-hss'}]}


## Ingest data to data stream

In [12]:
fake = Faker()
fake.add_provider(internet)
fake.add_provider(date_time)

logs = []
for _ in range(1_000):
    data = {
        'name': fake.name(),
        'address': fake.address(),
        'phone_number': fake.phone_number(),
        'email': fake.email(),
        'ip_address': fake.ipv4(),
        '@timestamp': fake.date_time_this_month()
    }
    logs.append({
        '_index': template_name,
        '_op_type': 'create',
        '_source': data
    })

bulk_result = helpers.bulk(client=client, actions=logs)
print(bulk_result)

(1000, [])


## Show data stream

In [13]:
data_stream_stats_result = client.indices.data_streams_stats(name=template_name)
print(data_stream_stats_result)

{'_shards': {'total': 2, 'successful': 2, 'failed': 0}, 'data_stream_count': 1, 'backing_indices': 1, 'total_store_size_bytes': 970516, 'data_streams': [{'data_stream': 'logs-hss', 'backing_indices': 1, 'store_size_bytes': 970516, 'maximum_timestamp': 1646848022000}]}


## Clean up

In [ ]:
client.indices.delete_index_template(template_name)
client.indices.delete_data_stream(template_name)